# Personagem

## Classe Personagem

In [ ]:
class Personagem():

    # globals()[name] = Personagem(name,pessoa,raca,classe,idade,nivel,antecedente,motivacao,modificacao,idiomas,gold,habil) ; players.append(globals()[name])
    def __init__(self,noun,people,race,clas,age,level,before,motivation,modification,languages,skills,religion = 'Panteao',animal_spirit = 'Nenhum'):

        # Infos do personagem.        
        self.nome , self.pessoa , self.raca , self.classe , self.idade , self.nivel = noun , people , dict_racas[race] , dict_classes[clas] , age , level
        self.antecedente , self.motivacao ,  self.modificacao , self.idiomas , self.dogma , self.espirito_animal = before , motivation , modification , languages , religion , animal_spirit

        # Vida e Mana iniciais.
        self.vida = self.mana = 60

        # Posses iniciais.
        self.peso , self.gold , self.itens, self.qtd_itens , self.corpo , self.maos = 0 , 500 , [soco_chute] , {'soco_chute' : 0} ,'nu' , [soco_chute,soco_chute]

        # Relação com os elementos.
        self.resistencias , self.vulnerabilidades , self.imunidades , self.absorcao = [] , [] , [] , []
        # Condições.
        self.qtd_dados , self.bonus_dado , self.bonus_dano , self.reduc_dano , self.condicao = 2 , 0 , 0 , 0 , []

        # Habilidades
        self.habilidades = skills.copy()

        # Atribui a raça ao personagem e os atributos consequentes.
        for row in dados['Racas'].iter_rows(min_col=1,min_row=2,max_col=33,max_row=2):
            for cell in row:
                if cell.value == self.raca:
                    self.habilidades.insert(0,cell.offset(row=0,column=1).value)
                    self.forca = cell.offset(row=1,column=1).value
                    self.agilidade = cell.offset(row=2,column=1).value
                    self.inteligencia = cell.offset(row=3,column=1).value
                    self.vontade = cell.offset(row=4,column=1).value

        # Atribui a classe ao personagem e os atributos consequentes.
        for row in dados['Classes'].iter_rows(min_col=1,min_row=2,max_col=22,max_row=2):
            for cell in row:
                if cell.value == self.classe:
                    self.habilidades.insert(0,cell.offset(row=0,column=1).value)
                    self.forca += cell.offset(row=1,column=1).value
                    self.agilidade += cell.offset(row=2,column=1).value
                    self.inteligencia += cell.offset(row=3,column=1).value
                    self.vontade += cell.offset(row=4,column=1).value      

        # Cálculo do atributos que dependem de outros.
        self.deslocamento = math.floor(self.agilidade/2) + 5 ; self.corrida = self.deslocamento*4
        if self.inteligencia >= self.vontade: self.determinacao = 8 + self.inteligencia
        else: self.determinacao = 8 + self.vontade 
        self.carga_basica = self.forca*7 ; self.carga_pesada = self.carga_basica*5 ; self.carga_maxima = self.carga_pesada*2
        self.esquiva = 5 + self.agilidade
        self.bloqueio = 5 + self.forca
        self.defesa = max(self.esquiva,self.bloqueio)

    def __repr__(self): return self.nome # Como o personagem vai aparecer se for impresso.

    def ataque(self,alvo): # Ataque normal do personagem.
        
        # Escolha do ataque.
        for i in range(len(self.maos)): print(i,'-',self.maos[i])
        escolha_ataque = input('Com qual arma deseja atacar? ')
        while escolha_ataque != '0' and escolha_ataque != '1':
            if escolha_ataque == 'sair': return
            escolha_ataque = input('Escolha uma arma que está na mão do personagem! ')
    
        # Verifica se pode atacar com a arma escolhida.
        while ('canalizador' in self.maos[int(escolha_ataque)].obs and self.maos[int(escolha_ataque)].dano == 0) or\
                ('ocupa_uma_mao' in self.maos[int(escolha_ataque)].obs) or ('som' in self.maos[int(escolha_ataque)].obs) :
                print('Não pode atacar com esse item.') ; escolha_ataque = input('Com qual arma deseja atacar? (digite "sair" caso não tenha arma para ataque) ')
                if escolha_ataque == 'sair': return

        atributo = 'força' ; atributo_ataque = self.forca # Atributo padrão para o ataque.

        # Verifica se é possível arremessar a arma.
        arremesso = ''
        if 'arremesso' in self.maos[int(escolha_ataque)].obs:
            arremesso = input('Deseja arremessar?(sim): ')
            while arremesso != 'sim' and arremesso != '':
                if arremesso == 'sair': return
                arremesso = input('Deseja arremessar?(Digite "sim", se não, deixe em branco): ')
            if arremesso == 'sim':
                atributo = 'agilidade' ; atributo_ataque = self.agilidade

        # Verifica se é necessário utilizar munição.
        elif hasattr(self.maos[int(escolha_ataque)],'municao'):
            for m in range (len(self.maos[int(escolha_ataque)].municao)): print(str(m + 1) + '-' + self.maos[int(escolha_ataque)].municao[m])
            # Escolha da munição.
            while True:
                try:
                    escolha_municao = input('Escolha uma munição da lista: ')
                    if escolha_municao == 'sair': return
                    escolha_municao = int(escolha_municao)
                except ValueError: print('Digite um número inteiro.'); continue
                else:
                    if escolha_municao < 1 and escolha_municao > len(self.maos[int(escolha_ataque)].municao): print('Escolha uma munição da lista.') ; continue
                    if self.maos[int(escolha_ataque)].municao[escolha_municao-1] not in self.qtd_itens: print('O personagem não possui esta munição.') ; continue
                    else: break
            atributo = 'agilidade' ; atributo_ataque = self.agilidade
            self.qtd_itens[self.maos[int(escolha_ataque)].municao[escolha_municao-1]] -= 1
            if self.qtd_itens[self.maos[int(escolha_ataque)].municao[escolha_municao-1]] == 0:
                self.itens.remove(globals()[self.maos[int(escolha_ataque)].municao[escolha_municao-1]])
                self.qtd_itens.pop(self.maos[int(escolha_ataque)].municao[escolha_municao-1])
                print(self.nome,'- Acabaram as',self.maos[int(escolha_ataque)].municao[escolha_municao-1])
            else: print(self.nome,'agora tem',self.qtd_itens[self.maos[int(escolha_ataque)].municao[escolha_municao-1]],self.maos[int(escolha_ataque)].municao[escolha_municao-1])
        print('\n')

        # Rolagem
        Di = atributo_ataque ; acct = 0 ; ecct = 0
        print('Atributo é',atributo,'=',atributo_ataque,',',Di)
        Di += self.bonus_dado
        print('Bonus de rolagem =',self.bonus_dado,',',Di)
        for d in range(self.qtd_dados):
            di = dado() ; Di += di ; print('Resultado do dado',d+1,'=',di,',',Di)
            if di == 6: acct += 1
            elif di == 1: ecct += 1
            if acct == 2 or ecct == 2: break
        print('Resultado Geral =',Di,' <> ','Defesa do alvo = ',alvo.defesa) ; forca = self.forca
        
        # Dano de acordo com resultado do dado.
        if self.maos[int(escolha_ataque)] == prodd or self.maos[int(escolha_ataque)] == besta_pesada or self.maos[int(escolha_ataque)] == besta_leve: self.forca = 0
        
        if Di >= alvo.defesa: print('Acertou!') ; damage = self.forca + self.maos[int(escolha_ataque)].dano + self.bonus_dano ;\
                                                receber_dano(alvo,damage,self.maos[int(escolha_ataque)].tipo,self)
        elif acct == 2: print('Acerto crítico!') ; damage = 2*(self.forca + self.maos[int(escolha_ataque)].dano + self.bonus_dano) ;\
                        receber_dano(alvo,damage,self.maos[int(escolha_ataque)].tipo,self)
        elif ecct == 2: print('Erro crítico!')
        else: print('Errou!')

        # reacao_geral(self)
        self.forca = forca
        if arremesso == 'sim':
            if ('duas_maos' in self.maos[int(escolha_ataque)].obs) or (self.forca < self.maos[int(escolha_ataque)].FN):
                if self.qtd_itens[self.maos[int(escolha_ataque)].nome] == 0: self.qtd_itens[self.maos[int(escolha_ataque)].nome] += 1 ; self.maos[0] = soco_chute ; self.maos[1] = soco_chute
                else: self.qtd_itens[self.maos[int(escolha_ataque)].nome] -= 1
            else:
                if self.qtd_itens[self.maos[int(escolha_ataque)].nome] == 0: self.maos[int(escolha_ataque)] = [soco_chute] ; self.qtd_itens[self.maos[int(escolha_ataque)].nome] += 1
                else: self.qtd_itens[self.maos[int(escolha_ataque)].nome] -= 1
        # return True!!!!!!!!
    def recalcular_atributos(self):
        self.deslocamento = math.floor(self.agilidade/2) + 5 ; self.corrida = self.deslocamento*4
        if self.corpo != 'nu':
            if hasattr(self.corpo,'rigida'): self.corrida = self.deslocamento*2
            elif hasattr(self.corpo,'pesada'): self.corrida = self.deslocamento*3
        if self.inteligencia >= self.vontade: self.determinacao = 8 + self.inteligencia
        else: self.determinacao = 8 + self.vontade     
        self.carga_basica = self.forca*7 ; self.carga_pesada = self.carga_basica*5 ; self.carga_maxima = self.carga_pesada*2 ; self.peso = 0
        self.esquiva = 5 + self.agilidade ; self.bloqueio = 5 + self.forca
        if self.corpo != 'nu': self.esquiva += self.corpo.defesa ; self.bloqueio += self.corpo.defesa
        if 'ocupa_uma_mao' in self.maos[0].obs: self.bloqueio += self.maos[0].defesa
        if 'ocupa_uma_mao' in self.maos[1].obs: self.bloqueio += self.maos[1].defesa
        self.defesa = max(self.esquiva,self.bloqueio)
        for i in self.itens: self.peso = i.peso*self.qtd_itens[i.nome]
        self.peso += self.maos[0].peso + self.maos[1].peso
        if self.corpo != 'nu': self.peso += self.corpo.peso
        if self.peso > self.carga_basica: self.deslocamento = self.deslocamento / 2 ; self.corrida = self.corrida / 2

    def equipar_maos(self):
        itens_maos = []
        for item in self.itens:
            if self.qtd_itens[item.nome] > 0: 
                if hasattr(item,'distancia'): itens_maos.append(item.nome)
                elif 'som' in item.obs: itens_maos.append(item.nome)
                elif 'ocupa_uma_mao' in item.obs: itens_maos.append(item.nome)
        print('maos =',self.maos); print('Itens para as mãos= ',itens_maos)
        hands = input('Qual ítem deseja equipar nas mãos? ("uma mao vazia" vai desequipar uma mao e "vazias" vai desequipar as duas mãos.) ')
        while hands not in itens_maos and hands != 'sair' and hands != 'ok' and hands != 'vazias' and hands != 'uma mao vazia':
            hands = input('Qual ÍTEM deseja equipar nas mãos? ')
        if hands == 'sair': return
        elif hands == 'vazias':
            if 'duas_maos' in self.maos[0].obs: self.qtd_itens[self.maos[0].nome] += 1
            elif self.maos[0] == self.maos[1] and self.forca < self.maos[1].FN: self.qtd_itens[self.maos[0].nome] += 1
            else: self.qtd_itens[self.maos[0].nome] += 1 ; self.qtd_itens[self.maos[1].nome] += 1
            self.maos[0] = soco_chute ; self.qtd_itens[soco_chute.nome] -= 1
            self.maos[1] = soco_chute ; self.qtd_itens[soco_chute.nome] -= 1
            print('maos= ',self.maos) ; self.recalcular_atributos()
        elif ('duas_maos' in globals()[hands].obs and self.forca >= globals()[hands].FN) or \
            ('duas_maos' not in globals()[hands].obs and 'ocupa_uma_mao' not in globals()[hands].obs and globals()[hands].FN / 2 <= self.forca < globals()[hands].FN):
            print('Essa arma vai substituir as duas mãos: ') ; print(self.maos) ; trocar = input('Deseja trocar?(sim \ nao) ')
            while trocar != 'sim' and trocar != 'nao': trocar = input('Deseja trocar?(sim OU nao) ')
            if trocar == 'sim':
                if 'duas_maos' in self.maos[0].obs: self.qtd_itens[self.maos[0].nome] += 1
                elif self.maos[0] == self.maos[1] and self.forca < self.maos[1].FN: self.qtd_itens[self.maos[0].nome] += 1
                else: self.qtd_itens[self.maos[0].nome] += 1 ; self.qtd_itens[self.maos[1].nome] += 1 
                self.maos[0] = globals()[hands] ; self.maos[1] = globals()[hands] ; self.qtd_itens[hands] -= 1
                print(self.maos) ; self.recalcular_atributos()
        elif self.forca >= globals()[hands].FN:
            print(self.maos); trocar = input('Deseja trocar por qual? (1,2,nao) ')
            while trocar != '1' and trocar != '2' and trocar != 'nao': trocar = input('Deseja trocar por qual, cacete?!?!? ')
            if trocar != 'nao':
                if 'duas_maos' in self.maos[0].obs: self.qtd_itens[self.maos[0].nome] += 1 ; self.maos[0] = self.maos[1] = soco_chute ; self.qtd_itens[soco_chute.nome] -= 2
                elif self.maos[0] == self.maos[1] and self.forca < self.maos[1].FN:
                    self.qtd_itens[self.maos[0].nome] += 1 ; self.maos[0] = self.maos[1] = soco_chute ; self.qtd_itens[soco_chute.nome] -= 2
                if trocar == '1':
                    self.qtd_itens[self.maos[0].nome] += 1 ; self.maos[0] = globals()[hands] ; self.qtd_itens[hands] -= 1 ; print(self.maos)
                if trocar == '2':
                    self.qtd_itens[self.maos[1].nome] += 1 ; self.maos[1] = globals()[hands] ; self.qtd_itens[hands] -= 1 ; print(self.maos)
                self.recalcular_atributos()
        else: print('Não é possível equipar este ítem.')    

    def equipar_corpo(self): #tirar arma dos itens quando equipar.
        itens_corpo = []
        for item in self.itens:
            if hasattr(item,'defesa') and 'ocupa_uma_mao' not in item.obs: itens_corpo.append(item.nome)
        print('Corpo = ',self.corpo); print('Itens para o corpo= ',itens_corpo)
        body = input('Qual armadura deseja equipar? ("nu" vai desequipar a armadura.): ')
        while body not in itens_corpo and body != 'sair' and body != 'nu': body = input('Qual ARMADURA deseja equipar? ')
        if body == 'sair' or (body == 'nu' and self.corpo == 'nu'): return
        elif body == 'nu' and self.corpo != 'nu': self.qtd_itens[self.corpo.nome] += 1 ; self.corpo = 'nu' ; self.recalcular_atributos()
        elif self.forca < globals()[body].FN: print('Não tem FN necessária para equipar este item.')
        elif self.forca >= globals()[body].FN:
            trocar = input('Deseja trocar?(sim \ nao) ')
            while trocar != 'sim' and trocar != 'nao': trocar = input('Deseja trocar?(sim OU nao, apenas) ')
            if trocar == 'sim': 
                if self.corpo != 'nu': self.qtd_itens[self.corpo.nome] += 1
                self.corpo = globals()[body] ; self.qtd_itens[body] -= 1 ; print('Armadura equipada=',self.corpo)
                self.recalcular_atributos()

    def vender_itens(self):
        while True:
            print(self.itens) ; print(self.qtd_itens) ; item_venda = input('Qual ítem deseja vender? ')
            while item_venda not in self.qtd_itens.keys() and item_venda != 'sair' or item_venda == 'soco_chute':
                item_venda = input('Quer ganhar dinheiro não? Qual ÍTEM deseja vender? ')
            if item_venda == 'sair': return
            if self.qtd_itens[item_venda] == 0: print('Tem que desequipar para vender.\n') ; continue
            while True:
                try: 
                    quantidade = input('Qual quantidade? ')
                    if quantidade == 'sair': return
                    quantidade = int(quantidade)
                except ValueError: print('Digite um número inteiro.') ; continue
                else:
                    if quantidade <= self.qtd_itens[item_venda]: break
                    else: print('Está tentando vender mais do que possui.')
            self.qtd_itens[item_venda] -= quantidade ; self.gold += quantidade*globals()[item_venda].custo
            print('Você ganhou ',quantidade*globals()[item_venda].custo,'de gold.')
            print('Você tem',self.gold,'de gold\n')
            if self.qtd_itens[item_venda] == 0:
                for venda in self.itens:
                    if venda.nome == item_venda: self.itens.remove(venda) ; self.qtd_itens.pop(venda.nome)

## Função para criar personagens

In [ ]:
def criar_personagem():

    print('Para sair digite: sair\n') # Para cancelar a criação do personagem, digite sair em qualquer input.

    # Nome do personagem
    name = input('Digite o nome do seu personagem: ')
    while name.isidentifier() is False or name in nomes_personagens:
        if name.isidentifier() is False: print('Nome inválido.')
        if name in nomes_personagens: print('Já há um personagem com esse nome.')
        name = input('\nDigite o nome do seu personagem: ')
    if name == 'sair': return print('Não vai jogar com a gente? :(')

    # Nome da Pessoa
    pessoa = input('\nQual o seu nome? ')
    while pessoa.isidentifier() is False or pessoa in nomes_personagens.values():
        if pessoa.isidentifier() is False: print('Tem nome não? Digite direito!')
        if pessoa in nomes_personagens.values(): print('Já há uma pessoa com esse nome.')
        pessoa = input('\nDigite o nome do seu personagem: ')
    if pessoa == 'sair': return print('Não vai jogar com a gente? :(')  

    # Escolha da Raça.
    print('\nLista de Raças.') ; print(dict_racas)
    raca = input('Digite a raça do seu personagem: ')
    while raca not in dict_racas and raca != 'sair': raca = input('Digite o NÚMERO com a raça do seu personagem: ')
    if raca == 'sair': return print('Não vai jogar com a gente? :(')
    print(dict_racas[raca])

    # Escolha da Classe.
    print('\nLista de Classes.') ; print(dict_classes)
    classe = input('Digite a classe do seu personagem: ')
    while classe not in dict_classes and classe!='sair': classe = input('Digite o NÚMERO com a CLASSE do seu personagem: ')
    if classe == 'sair': return print('Não vai jogar com a gente? :(')
    print(dict_classes[classe])

    # Idade.
    while True:
        try:
            idade = input('\nDigite a idade do seu personagem: ')
            if idade == 'sair': return print('Não vai jogar com a gente? :(')
            idade = int(idade)
        except ValueError: print('Digite um número inteiro.') ; continue
        else: break

    #Nível.
    while True:
        try:
            nivel = input('Digite o nível do seu personagem: ')
            if nivel == 'sair': return print('Não vai jogar com a gente? :(')
            nivel = int(nivel)
        except ValueError: print('Digite um número inteiro de 1 a 10.'); continue
        else:
            if nivel < 1 or nivel > 10: print('Digite um número inteiro de 1 a 10.') ; continue
            else: break

    #Antecedente.
    antecedente = input('Digite o antecedente do seu personagem (se não tiver, deixar em branco): ')
    if antecedente == 'sair': return print('Não vai jogar com a gente? :(')

    #Motivacao
    motivacao = input('Escreva a motivação do seu personagem (se não tiver, deixar em branco): ')
    if motivacao == 'sair': return print('Não vai jogar com a gente? :(')

    #Modificacao
    modificacao = input('Digite a modificação do seu personagem (se não tiver, deixar em branco): ')
    if modificacao == 'sair': return print('Não vai jogar com a gente? :(')

    #Idiomas
    while True:
        try:
            n_idiomas = input('Digite o número de idiomas que o seu personagem fala: ')
            if n_idiomas == 'sair': return print('Não vai jogar com a gente? :(')
            n_idiomas = int(n_idiomas)
        except ValueError: print('Digite um número inteiro.') ; continue
        else: break    
    idiomas = []
    for n in range(n_idiomas):
        idioma = input('Idioma ' + str(n + 1) + ': ')
        if idioma == 'sair': return print('Não vai jogar com a gente? :(')
        idiomas.append(idioma)
    
    
    #Habilidades
    print('\n') ; habil = []
    print('Escolha 3 habilidades')
    for h in range(3): 
        skills = input('Habilidade ' + str(h+1) + ': ')
        if skills == 'sair': return print('Não vai jogar com a gente? :(')
        else: habil.append(skills)

    # Dogma
    dogma = input('Digite o do dogma seu personagem (se não tiver, deixar em branco): ')
    if dogma == 'sair': return print('Não vai jogar com a gente? :(')
    
    # Espírito Animal
    espirito_animal= input('Digite o espírito animal do seu personagem (se não tiver, deixar em branco): ')
    if espirito_animal == 'sair': return print('Não vai jogar com a gente? :(')

    # Cria o personagem
    nomes_personagens[name] = pessoa
    print('Surge mais um herói!')
    globals()[name] = Personagem(name,pessoa,raca,classe,idade,nivel,antecedente,motivacao,modificacao,idiomas,habil,dogma,espirito_animal) ; players.append(globals()[name])